In [1]:
import org.apache.spark.sql.SparkSession

In [2]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@21fa9afe


org.apache.spark.sql.SparkSession@21fa9afe

In [3]:
spark.sql("use raj")

[]

In [4]:
import spark.implicits._

In [5]:
import org.apache.spark.sql.functions._

In [6]:
import org.apache.spark.sql.Column

#### In order to use the windowing functions in spark dataframe we need to import the below package

In [7]:
import org.apache.spark.sql.expressions._

In [12]:
val df=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders/part-00000-6502fdf1-89f6-48b4-ae4c-6c0376ab98ce-c000.snappy.parquet")

df = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [19]:
df.withColumn("Rank",rank().over(Window.partitionBy("Order_date").orderBy("order_id"))).show()

+--------+-------------------+----------------+---------------+----+
|order_id|         Order_date|Order_product_id|   Order_status|Rank|
+--------+-------------------+----------------+---------------+----+
|   51203|2014-06-11 00:00:00|            5894|PENDING_PAYMENT|   1|
|   51204|2014-06-11 00:00:00|            2734|PENDING_PAYMENT|   2|
|   51205|2014-06-11 00:00:00|           11652|PENDING_PAYMENT|   3|
|   51206|2014-06-11 00:00:00|           10555|PENDING_PAYMENT|   4|
|   51207|2014-06-11 00:00:00|            8024|PENDING_PAYMENT|   5|
|   51208|2014-06-11 00:00:00|            2691|       COMPLETE|   6|
|   51209|2014-06-11 00:00:00|            7705|       COMPLETE|   7|
|   51210|2014-06-11 00:00:00|            8180|        ON_HOLD|   8|
|   51211|2014-06-11 00:00:00|            5955|         CLOSED|   9|
|   51212|2014-06-11 00:00:00|           10514|       COMPLETE|  10|
|   51213|2014-06-11 00:00:00|            7445|        ON_HOLD|  11|
|   51214|2014-06-11 00:00:00|    

In [20]:
val df1=spark.read.schema("order_item_id int,order_item_order_id int,order_item_product_id int,order_item_quantity int,order_item_subtotal decimal(17,4),order_item_product_price decimal(17,4)").csv("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_item_new")

df1 = [order_item_id: int, order_item_order_id: int ... 4 more fields]


[order_item_id: int, order_item_order_id: int ... 4 more fields]

In [23]:
val orderItem=df1.drop("order_item_quantity","order_item_product_id","order_item_product_price")

orderItem = [order_item_id: int, order_item_order_id: int ... 1 more field]


[order_item_id: int, order_item_order_id: int ... 1 more field]

In [24]:
orderItem.printSchema()

root
 |-- order_item_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_subtotal: decimal(17,4) (nullable = true)



In [25]:
val spec=Window.partitionBy("order_item_order_id")

spec = org.apache.spark.sql.expressions.WindowSpec@4f2dcdd


org.apache.spark.sql.expressions.WindowSpec@4f2dcdd

In [30]:
val orderItemRevenue=orderItem.withColumn("order_Revenue",sum("order_item_subtotal").over(spec))

orderItemRevenue = [order_item_id: int, order_item_order_id: int ... 2 more fields]


[order_item_id: int, order_item_order_id: int ... 2 more fields]

In [27]:
orderItemRevenue.printSchema()

root
 |-- order_item_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_subtotal: decimal(17,4) (nullable = true)
 |-- order_Revenue: decimal(27,4) (nullable = true)



In [31]:
orderItemRevenue.sort("order_item_order_id").show()

+-------------+-------------------+-------------------+-------------+
|order_item_id|order_item_order_id|order_item_subtotal|order_Revenue|
+-------------+-------------------+-------------------+-------------+
|            1|                  1|           299.9800|     299.9800|
|            3|                  2|           250.0000|     579.9800|
|            4|                  2|           129.9900|     579.9800|
|            2|                  2|           199.9900|     579.9800|
|            8|                  4|           199.9200|     699.8500|
|            7|                  4|           150.0000|     699.8500|
|            6|                  4|           299.9500|     699.8500|
|            5|                  4|            49.9800|     699.8500|
|           10|                  5|           299.9500|    1129.8600|
|           12|                  5|           299.9800|    1129.8600|
|            9|                  5|           299.9800|    1129.8600|
|           13|     

In [8]:
val df2=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/emp_join")

df2 = [EMPNO: int, ENAME: string ... 6 more fields]


[EMPNO: int, ENAME: string ... 6 more fields]

In [9]:
spark.conf.set("spark.sql.shuffle.partitions","2")

In [10]:
val spec=Window.partitionBy("DEPTNO").orderBy($"SAL"desc)

spec = org.apache.spark.sql.expressions.WindowSpec@33ea51f1


org.apache.spark.sql.expressions.WindowSpec@33ea51f1

In [11]:
val emp_sal=df2.select("EMPNO","DEPTNO","SAL").withColumn("total_sal",sum("SAL").over(spec)).orderBy($"DEPTNO",$"SAL".desc)

emp_sal = [EMPNO: int, DEPTNO: int ... 2 more fields]


[EMPNO: int, DEPTNO: int ... 2 more fields]

In [69]:
emp_sal.show()

+-----+------+----+---------+
|EMPNO|DEPTNO| SAL|total_sal|
+-----+------+----+---------+
| 7839|    10|5000|     8750|
| 7782|    10|2450|     8750|
| 7934|    10|1300|     8750|
| 7902|    20|3000|    10875|
| 7788|    20|3000|    10875|
| 7566|    20|2975|    10875|
| 7876|    20|1100|    10875|
| 7369|    20| 800|    10875|
| 7698|    30|2850|     9400|
| 7499|    30|1600|     9400|
| 7844|    30|1500|     9400|
| 7654|    30|1250|     9400|
| 7521|    30|1250|     9400|
| 7900|    30| 950|     9400|
+-----+------+----+---------+



In [14]:
emp_sal.groupBy("DEPTNO").avg("SAL").show()

+------+------------------+
|DEPTNO|          avg(SAL)|
+------+------------------+
|    10|2916.6666666666665|
|    20|            2175.0|
|    30|1566.6666666666667|
+------+------------------+



In [18]:
val emp_avg_sal=df2.withColumn("Avg_sal",avg("SAL").over(spec)).orderBy($"DEPTNO",$"SAL".desc).show()

+-----+------+---------+----+----------+----+----+------+------------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|           Avg_sal|
+-----+------+---------+----+----------+----+----+------+------------------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|2916.6666666666665|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|2916.6666666666665|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|2916.6666666666665|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|            2175.0|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|            2175.0|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|            2175.0|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|            2175.0|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|            2175.0|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|1566.6666666666667|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|1566.6666666666667|

emp_avg_sal: Unit = ()


In [30]:
val emp_lead=df2.select("EMPNO","DEPTNO","SAL").withColumn("emp_lead",lead("SAL",1).over(spec)).show()

+-----+------+----+--------+
|EMPNO|DEPTNO| SAL|emp_lead|
+-----+------+----+--------+
| 7839|    10|5000|    2450|
| 7782|    10|2450|    1300|
| 7934|    10|1300|    null|
| 7788|    20|3000|    3000|
| 7902|    20|3000|    2975|
| 7566|    20|2975|    1100|
| 7876|    20|1100|     800|
| 7369|    20| 800|    null|
| 7698|    30|2850|    1600|
| 7499|    30|1600|    1500|
| 7844|    30|1500|    1250|
| 7521|    30|1250|    1250|
| 7654|    30|1250|     950|
| 7900|    30| 950|    null|
+-----+------+----+--------+



emp_lead: Unit = ()


In [32]:
val emp_lead=df2.select("EMPNO","DEPTNO","SAL").withColumn("emp_lead",lead("SAL",2).over(spec)).show()

+-----+------+----+--------+
|EMPNO|DEPTNO| SAL|emp_lead|
+-----+------+----+--------+
| 7839|    10|5000|    1300|
| 7782|    10|2450|    null|
| 7934|    10|1300|    null|
| 7788|    20|3000|    2975|
| 7902|    20|3000|    1100|
| 7566|    20|2975|     800|
| 7876|    20|1100|    null|
| 7369|    20| 800|    null|
| 7698|    30|2850|    1500|
| 7499|    30|1600|    1250|
| 7844|    30|1500|    1250|
| 7521|    30|1250|     950|
| 7654|    30|1250|    null|
| 7900|    30| 950|    null|
+-----+------+----+--------+



emp_lead: Unit = ()


In [45]:
val emp_1=df2.select("EMPNO","DEPTNO","SAL").withColumn("emp_lag",lag("SAL",1).over(spec)).show()

+-----+------+----+-------+
|EMPNO|DEPTNO| SAL|emp_lag|
+-----+------+----+-------+
| 7839|    10|5000|   null|
| 7782|    10|2450|   5000|
| 7934|    10|1300|   2450|
| 7788|    20|3000|   null|
| 7902|    20|3000|   3000|
| 7566|    20|2975|   3000|
| 7876|    20|1100|   2975|
| 7369|    20| 800|   1100|
| 7698|    30|2850|   null|
| 7499|    30|1600|   2850|
| 7844|    30|1500|   1600|
| 7521|    30|1250|   1500|
| 7654|    30|1250|   1250|
| 7900|    30| 950|   1250|
+-----+------+----+-------+



emp_1: Unit = ()


In [44]:
df2.select("EMPNO","DEPTNO","SAL").withColumn("First_Sal",first("SAL").over(spec)).show()

+-----+------+----+---------+
|EMPNO|DEPTNO| SAL|First_Sal|
+-----+------+----+---------+
| 7839|    10|5000|     5000|
| 7782|    10|2450|     5000|
| 7934|    10|1300|     5000|
| 7788|    20|3000|     3000|
| 7902|    20|3000|     3000|
| 7566|    20|2975|     3000|
| 7876|    20|1100|     3000|
| 7369|    20| 800|     3000|
| 7698|    30|2850|     2850|
| 7499|    30|1600|     2850|
| 7844|    30|1500|     2850|
| 7521|    30|1250|     2850|
| 7654|    30|1250|     2850|
| 7900|    30| 950|     2850|
+-----+------+----+---------+



In [14]:
val spec=Window.partitionBy("DEPTNO").orderBy($"SAL".desc).rangeBetween(unboundedPreceding,unboundedFollowing)

spec = org.apache.spark.sql.expressions.WindowSpec@6a72497f


org.apache.spark.sql.expressions.WindowSpec@6a72497f

In [16]:
df2.select("EMPNO","DEPTNO","SAL").withColumn("Last_Sal",last("SAL",false).over(spec)).orderBy($"DEPTNO",$"SAL".desc).show()

+-----+------+----+--------+
|EMPNO|DEPTNO| SAL|Last_Sal|
+-----+------+----+--------+
| 7839|    10|5000|    1300|
| 7782|    10|2450|    1300|
| 7934|    10|1300|    1300|
| 7788|    20|3000|     800|
| 7902|    20|3000|     800|
| 7566|    20|2975|     800|
| 7876|    20|1100|     800|
| 7369|    20| 800|     800|
| 7698|    30|2850|     950|
| 7499|    30|1600|     950|
| 7844|    30|1500|     950|
| 7521|    30|1250|     950|
| 7654|    30|1250|     950|
| 7900|    30| 950|     950|
+-----+------+----+--------+



In [17]:
val spec=Window.partitionBy("DEPTNO").orderBy($"SAL".desc)

spec = org.apache.spark.sql.expressions.WindowSpec@16867486


org.apache.spark.sql.expressions.WindowSpec@16867486

In [23]:
df2.select("EMPNO","DEPTNO","SAL").withColumn("rank",rank.over(spec)).orderBy($"DEPTNO",$"SAL".desc).show()

+-----+------+----+----+
|EMPNO|DEPTNO| SAL|rank|
+-----+------+----+----+
| 7839|    10|5000|   1|
| 7782|    10|2450|   2|
| 7934|    10|1300|   3|
| 7788|    20|3000|   1|
| 7902|    20|3000|   1|
| 7566|    20|2975|   3|
| 7876|    20|1100|   4|
| 7369|    20| 800|   5|
| 7698|    30|2850|   1|
| 7499|    30|1600|   2|
| 7844|    30|1500|   3|
| 7521|    30|1250|   4|
| 7654|    30|1250|   4|
| 7900|    30| 950|   6|
+-----+------+----+----+



In [24]:
df2.select("EMPNO","DEPTNO","SAL").withColumn("dense_rank",dense_rank.over(spec)).orderBy($"DEPTNO",$"SAL".desc).show()

+-----+------+----+----------+
|EMPNO|DEPTNO| SAL|dense_rank|
+-----+------+----+----------+
| 7839|    10|5000|         1|
| 7782|    10|2450|         2|
| 7934|    10|1300|         3|
| 7788|    20|3000|         1|
| 7902|    20|3000|         1|
| 7566|    20|2975|         2|
| 7876|    20|1100|         3|
| 7369|    20| 800|         4|
| 7698|    30|2850|         1|
| 7499|    30|1600|         2|
| 7844|    30|1500|         3|
| 7521|    30|1250|         4|
| 7654|    30|1250|         4|
| 7900|    30| 950|         5|
+-----+------+----+----------+



In [25]:
df2.select("EMPNO","DEPTNO","SAL").withColumn("Rownumber",row_number.over(spec)).orderBy($"DEPTNO",$"SAL".desc).show()

+-----+------+----+---------+
|EMPNO|DEPTNO| SAL|Rownumber|
+-----+------+----+---------+
| 7839|    10|5000|        1|
| 7782|    10|2450|        2|
| 7934|    10|1300|        3|
| 7788|    20|3000|        1|
| 7902|    20|3000|        2|
| 7566|    20|2975|        3|
| 7876|    20|1100|        4|
| 7369|    20| 800|        5|
| 7698|    30|2850|        1|
| 7499|    30|1600|        2|
| 7844|    30|1500|        3|
| 7521|    30|1250|        4|
| 7654|    30|1250|        5|
| 7900|    30| 950|        6|
+-----+------+----+---------+

